In [1]:
#!/usr/bin/python

import sys
import pickle
import pandas as pd
import numpy
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

/Users/benjaminxiao/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
# features_list is a list of my selected features
# finance_features has finance features I'm interested in
# email_features is a list of email features
features_list = ['poi', 'bonus', 'expenses', 'bon_plus_expenses', \
                'to_msg_ratio', 'from_msg_ratio']
finance_features = ['poi', 'salary', 'bonus', 'long_term_incentive', \
                 'deferred_income', 'expenses', 'total_payments', \
                 'exercised_stock_options', 'restricted_stock', 'other'] 
email_features = ['poi', 'to_messages', 'email_address', 
                 'from_poi_to_this_person', 'from_messages', \
                 'from_this_person_to_poi', 'shared_receipt_with_poi', \
                 'to_msg_ratio', 'from_msg_ratio']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

df = pd.DataFrame.from_records(list(data_dict.values()))
employees = pd.Series(list(data_dict.keys()))

# set the index of df to be the employees series:
df.set_index(employees, inplace=True)

In [3]:
# Find how many missing values are in my selected features
for feature in features_list:
    try:
        print "Number of missing values in " + str(feature) + ": " + str(df[feature].value_counts(dropna=False)[0])
    except:
        print "Created feature: ", str(feature)

Number of missing values in poi: 128
Number of missing values in bonus: 64
Number of missing values in expenses: 51
Created feature:  bon_plus_expenses
Created feature:  to_msg_ratio
Created feature:  from_msg_ratio


In [4]:
# Create another working dataframe to make new features 

df_new = df.apply(lambda x: pd.to_numeric(x, errors='coerce')).copy()

df_new

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
METTS MARK,600000.0,NaN,NaN,NaN,NaN,NaN,94299.0,29.0,38.0,1.0,...,NaN,1740.0,False,585062.0,NaN,365788.0,702.0,807.0,1061827.0,585062.0
BAXTER JOHN C,1200000.0,1295738.0,-1386055.0,NaN,NaN,6680544.0,11200.0,NaN,NaN,NaN,...,1586055.0,2660303.0,False,3942714.0,NaN,267102.0,NaN,NaN,5634343.0,10623258.0
ELLIOTT STEVEN,350000.0,NaN,-400729.0,NaN,NaN,4890344.0,78552.0,NaN,NaN,NaN,...,NaN,12961.0,False,1788391.0,NaN,170941.0,NaN,NaN,211725.0,6678735.0
CORDES WILLIAM R,NaN,NaN,NaN,NaN,NaN,651850.0,NaN,12.0,10.0,0.0,...,NaN,NaN,False,386335.0,NaN,NaN,58.0,764.0,NaN,1038185.0
HANNON KEVIN P,1500000.0,NaN,-3117011.0,NaN,NaN,5538001.0,34039.0,32.0,32.0,21.0,...,1617011.0,11350.0,True,853064.0,NaN,243293.0,1035.0,1045.0,288682.0,6391065.0
MORDAUNT KRISTINA M,325000.0,NaN,NaN,NaN,NaN,NaN,35018.0,NaN,NaN,NaN,...,NaN,1411.0,False,208510.0,NaN,267093.0,NaN,NaN,628522.0,208510.0
MEYER ROCKFORD G,NaN,1848227.0,NaN,NaN,NaN,493489.0,NaN,28.0,0.0,0.0,...,NaN,NaN,False,462384.0,NaN,NaN,22.0,232.0,1848227.0,955873.0
MCMAHON JEFFREY,2600000.0,NaN,NaN,NaN,NaN,1104054.0,137108.0,48.0,58.0,26.0,...,694862.0,297353.0,False,558801.0,NaN,370448.0,2228.0,2355.0,4099771.0,1662855.0
HORTON STANLEY C,NaN,3131860.0,NaN,NaN,NaN,5210569.0,NaN,1073.0,44.0,15.0,...,NaN,NaN,False,2046079.0,NaN,NaN,1074.0,2350.0,3131860.0,7256648.0
PIPER GREGORY F,400000.0,1130036.0,-33333.0,NaN,NaN,880290.0,43057.0,222.0,61.0,48.0,...,NaN,778.0,False,409554.0,-409554.0,197091.0,742.0,1238.0,1737629.0,880290.0


In [5]:
# from_msg_ratio is ratio messages received from poi to total messages received
df_new['to_msg_ratio'] = df_new.from_this_person_to_poi.divide(df_new.to_messages, axis = 'index')
# create to_msg_ratio by dividing from_this_person_to_poi from to_messages
df_new['from_msg_ratio'] = df_new.from_poi_to_this_person.divide(df_new.from_messages, axis = 'index')
# create a new feature by adding expenses and bonus together
df_new['bon_plus_expenses'] = df_new['bonus'].add(df_new['expenses'], axis = 'index')

In [6]:
# Fill NaN with 0 where operations created NaN in some rows
df_new.fillna(0, inplace = True)

In [7]:
#df_new['bon_plus_expenses']

In [8]:
# create a dictionary from the dataframe
df_dict = df_new.to_dict('index')

In [9]:
# after you create features, the column names will be your new features
# create a list of column names:
new_features_list = df_new.columns.values
new_features_list

array(['bonus', 'deferral_payments', 'deferred_income', 'director_fees',
       'email_address', 'exercised_stock_options', 'expenses',
       'from_messages', 'from_poi_to_this_person',
       'from_this_person_to_poi', 'loan_advances', 'long_term_incentive',
       'other', 'poi', 'restricted_stock', 'restricted_stock_deferred',
       'salary', 'shared_receipt_with_poi', 'to_messages',
       'total_payments', 'total_stock_value', 'to_msg_ratio',
       'from_msg_ratio', 'bon_plus_expenses'], dtype=object)

In [10]:
### Task 2: Remove outliers

# I want to try out classifiers first before removing outliers to see 
# if it would perform any better.

In [11]:
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = df_dict

In [12]:
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [13]:
### Extract features and labels from dataset for local testing
# Use list finances_features
data = featureFormat(my_dataset, finance_features, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [14]:
### Extract features and labels from dataset for local testing
# Same as before, but uses list email_features
data = featureFormat(my_dataset, email_features, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [15]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.



In [27]:
# Trying PCA to combine some features

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(features)


#print pca.explained_variance_ratio_
print pca.components_
#print pca.explained_variance_

[[  8.56956148e-01  -0.00000000e+00   1.55514190e-02   3.95535731e-01
    2.15313366e-02   3.29351177e-01   1.30341064e-06  -5.72035959e-05]
 [ -3.01750918e-01  -1.11022302e-16  -1.17372225e-02   9.04316378e-01
    1.49591627e-02  -3.01324892e-01   4.59513264e-06  -2.05440483e-04]
 [  4.17723446e-01  -1.11022302e-16  -5.90009853e-02  -1.58362882e-01
   -2.29593196e-02  -8.92423267e-01  -6.81539672e-06  -8.86285125e-04]
 [  2.63022735e-04   1.61979805e-16   5.10867864e-01  -2.45836542e-02
    8.57769356e-01  -5.13592191e-02   2.19554400e-04   1.87477876e-03]
 [ -8.91924682e-03  -1.07552856e-16  -8.57279904e-01  -8.95620793e-03
    5.12807780e-01   4.09151519e-02   1.89519169e-04  -1.62940134e-02]
 [  2.11456857e-04  -5.67254577e-16  -1.49821233e-02  -3.17931955e-05
    6.73239964e-03  -7.10563007e-05   1.22161929e-04   9.99865063e-01]
 [  4.72335944e-06   2.86771331e-13   5.17694847e-05   1.34840147e-06
   -2.86589678e-04  -1.59625335e-06   9.99999950e-01  -1.19474061e-04]
 [  4.9078019

In [17]:
features_train = pca.transform(features)

In [26]:
# First one tried is RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf = clf.fit(features, labels)

print clf.feature_importances_

[ 0.10525018  0.          0.11835894  0.13819868  0.18258433  0.17153572
  0.16748446  0.11658769]


In [19]:
# Next, Adaboost
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()
clf = clf.fit(features, labels)

print clf.feature_importances_

[ 0.14  0.    0.06  0.04  0.06  0.28  0.26  0.16]


In [20]:
# Check accuracy scores
#print clf.score(features)

TypeError: score() takes at least 3 arguments (2 given)

In [21]:
print clf.feature_importances_[4]
print email_features[4]

0.06
from_messages


In [22]:
print finance_features[1:]

# bonus and exercised stock option are two more important ones
# also expenses

['salary', 'bonus', 'long_term_incentive', 'deferred_income', 'expenses', 'total_payments', 'exercised_stock_options', 'restricted_stock', 'other']


In [23]:
print email_features[1:]

# to_messages, shared_receipt_with_poi are both best
# the two msg ratios aren't that great, so work with shared receipt

['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi', 'to_msg_ratio', 'from_msg_ratio']


In [24]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info:
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)
